In [3]:
#cnn2.py - grad2020 - Testado em TF2 em Colab
import os; os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau
import numpy as np; import sys
def deslocaEsquerda(a):
  d=a.copy(); d[:,0:-1]=a[:,1:]; return d
def deslocaDireita(a):
  d=a.copy(); d[:,1:]=a[:,0:-1]; return d
def deslocaCima(a):
  d=a.copy(); d[0:-1,:]=a[1:,:]; return d
def deslocaBaixo(a):
  d=a.copy(); d[1:,:]=a[0:-1,:]; return d
print("Lendo MNIST")


Lendo MNIST


In [7]:
(AX, AY), (QX, QY) = mnist.load_data()
AX=255-AX; QX=255-QX

AX = np.resize(AX,(5*60000,28,28))
AY = np.resize(AY,(5*60000,1))
for s in range(60000):
  AX[s+60000]=deslocaEsquerda(AX[s])
  AX[s+2*60000]=deslocaDireita(AX[s])
  AX[s+3*60000]=deslocaCima(AX[s])
  AX[s+4*60000]=deslocaBaixo(AX[s])
  AY[s+60000]=AY[s]
  AY[s+2*60000]=AY[s]
  AY[s+3*60000]=AY[s]
  AY[s+4*60000]=AY[s]

In [8]:
print("Convertendo para categorico e float")
nclasses = 10
AY2 = keras.utils.to_categorical(AY, nclasses)
QY2 = keras.utils.to_categorical(QY, nclasses)
nl, nc = AX.shape[1], AX.shape[2] #28, 28
AX = AX.astype('float32') / 255.0 - 0.5 # -0.5 a +0.5
QX = QX.astype('float32') / 255.0 - 0.5 # -0.5 a +0.5
AX = AX.reshape(AX.shape[0], nl, nc, 1)
QX = QX.reshape(QX.shape[0], nl, nc, 1)

Convertendo para categorico e float


In [11]:
model = Sequential()
model.add(Conv2D(20, kernel_size=(3,3), activation='relu', input_shape=(nl,nc,1))) #28 - 3 +1 = 26
model.add(MaxPooling2D(pool_size=(2,2))) #13
model.add(Conv2D(40, kernel_size=(3,3), activation='relu')) #13 - 3 +1 = 1
model.add(MaxPooling2D(pool_size=(2,2))) #7
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(60, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nclasses, activation='softmax'))
print("Treinando modelo")
opt=optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])
reduce_lr = ReduceLROnPlateau(monitor='accuracy',
 factor=0.9, patience=2, min_lr=0.0001, verbose=True)
model.fit(AX, AY2, batch_size=1000, epochs=100, verbose=2,
 validation_data=(QX, QY2), callbacks=[reduce_lr])
score = model.evaluate(QX, QY2, verbose=False)
print('Test loss: %.4f'%(score[0]))
print('Test accuracy: %.2f %%'%(100*score[1]))
print('Test error: %.2f %%'%(100*(1-score[1])))
model.save("cnn2.h5")


Treinando modelo
Epoch 1/100
300/300 - 142s - loss: 0.4269 - accuracy: 0.8656 - val_loss: 0.0448 - val_accuracy: 0.9860 - lr: 0.0010 - 142s/epoch - 475ms/step
Epoch 2/100
300/300 - 142s - loss: 0.1114 - accuracy: 0.9701 - val_loss: 0.0303 - val_accuracy: 0.9905 - lr: 0.0010 - 142s/epoch - 472ms/step
Epoch 3/100
300/300 - 140s - loss: 0.0797 - accuracy: 0.9791 - val_loss: 0.0241 - val_accuracy: 0.9926 - lr: 0.0010 - 140s/epoch - 468ms/step
Epoch 4/100
300/300 - 141s - loss: 0.0644 - accuracy: 0.9833 - val_loss: 0.0249 - val_accuracy: 0.9919 - lr: 0.0010 - 141s/epoch - 469ms/step
Epoch 5/100
300/300 - 141s - loss: 0.0558 - accuracy: 0.9855 - val_loss: 0.0194 - val_accuracy: 0.9939 - lr: 0.0010 - 141s/epoch - 470ms/step
Epoch 6/100
300/300 - 140s - loss: 0.0495 - accuracy: 0.9870 - val_loss: 0.0164 - val_accuracy: 0.9949 - lr: 0.0010 - 140s/epoch - 466ms/step
Epoch 7/100
300/300 - 139s - loss: 0.0437 - accuracy: 0.9884 - val_loss: 0.0220 - val_accuracy: 0.9938 - lr: 0.0010 - 139s/epoch - 

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt

mean_grey = 0.1307
stdd_gray = 0.3081

transforms = transforms.Compose([transforms.ToTensor(),transforms.Normalize((mean_grey),(stdd_gray))])

train_dataset = datasets.MNIST(root='./data',
                           train=True,
                           transform=transforms,
                           download=True)
test_dataset = datasets.MNIST(root='./data',
                           train=False,
                           transform=transforms)



In [2]:
img = (train_dataset[20][0].numpy())
#plt.imshow(img.reshape(28,28), cmap = 'gray')

In [3]:
print(img.reshape(28,28))

[[-0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
  -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
  -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
  -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
  -0.42421296 -0.42421296 -0.42421296 -0.42421296]
 [-0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
  -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
  -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
  -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
  -0.42421296 -0.42421296 -0.42421296 -0.42421296]
 [-0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
  -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
  -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
  -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
  -0.42421

In [4]:
batch_size = 100   

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

print(len(train_dataset))
print(len(train_loader))

60000
600


In [5]:
class CNN(nn.Module):
    def __init__(self, kernel_size_1):
        super(CNN,self).__init__()
        #output size = (in - kernel_size + 2*padding)/stride + 1 = 28- 3 +2*1 +1 =28
        self.cnn1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = kernel_size_1,stride = 1, padding = 1)
        self.batchnorm = nn.BatchNorm2d(8)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size = 2)
        self.cnn2 = nn.Conv2d(in_channels = 8, out_channels = 32, kernel_size = 5,stride = 1, padding = 2)
        self.batchnorm2 = nn.BatchNorm2d(32)
        #32x7x7 = 1568
        self.fc1 = nn.Linear(1568,600)
        self.dropout = nn.Dropout(p = 0.5)
        self.fc2 = nn.Linear(600,10)
    def forward(self, x): 
        x1 = self.cnn1(x)
        x1 = self.batchnorm(x1)
        x1 = self.relu(x1)
        x1 = self.maxpool(x1)
        
        x2 = self.cnn2(x1)
        x2 = self.batchnorm2(x2)
        x2 = self.relu(x2)
        x2 = self.maxpool(x2)
        #flatten
        x2 = x2.view(batch_size, 1568)
        
        out = self.fc1(x2)
        out = self.dropout(out)
        out = self.fc2(out)
        return out

In [6]:
model = CNN(kernel_size_1 = 3)
CUDA = torch.cuda.is_available()
if CUDA:
    model = model.cuda()
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [7]:
epochs = 10 
train_loss = []
train_accuracy = []
test_loss = []
test_accuracy = []

for epoch in range(epochs):
    correct = 0
    iterations = 0
    iter_loss = 0
    model.train()
    
    for i, (inputs, labels) in enumerate(train_loader):
        if CUDA:
            inputs = inputs.cuda()
            labels = labels.cuda()
        outputs = model(inputs)
        loss = loss_func(outputs, labels)                 # Difference between the actual and predicted (loss function)
        iter_loss += loss.item()
        optimizer.zero_grad() 
        loss.backward()                                   # Backpropagation
        optimizer.step()  
        
        _, predicted = torch.max(outputs, 1) 
        correct += (predicted == labels).sum().item()
        iterations += 1
    train_loss.append(iter_loss/iterations)
    train_accuracy.append(100*correct/len(train_dataset))
    
    testing_loss = 0.0
    correct = 0.0
    iterations = 0.0
    
    model.eval()
    for i, (inputs, labels) in enumerate(test_loader):
        if CUDA:
            inputs = inputs.cuda()
            labels = labels.cuda()
        outputs = model(inputs)
        loss = loss_func(outputs, labels)                 # Difference between the actual and predicted (loss function)
        testing_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1) 
        correct += (predicted == labels).sum().item()
        iterations += 1
    test_loss.append(testing_loss/iterations)
    test_accuracy.append(100*correct/len(test_dataset))
    
    print('Epoch [{}/{}], Training Loss: {:.3f}, Training Accuracy: {:.3f}%, Test Loss: {:.3f}, Test Accuracy: {:.3f}%'.format
          (epoch+1, epochs, train_loss[-1], train_accuracy[-1], test_loss[-1], test_accuracy[-1]))

Epoch [1/10], Training Loss: 0.835, Training Accuracy: 93.127%, Test Loss: 0.061, Test Accuracy: 97.940%
Epoch [2/10], Training Loss: 0.065, Training Accuracy: 98.073%, Test Loss: 0.051, Test Accuracy: 98.450%
Epoch [3/10], Training Loss: 0.053, Training Accuracy: 98.303%, Test Loss: 0.037, Test Accuracy: 98.890%
Epoch [4/10], Training Loss: 0.049, Training Accuracy: 98.500%, Test Loss: 0.044, Test Accuracy: 98.670%
Epoch [5/10], Training Loss: 0.052, Training Accuracy: 98.442%, Test Loss: 0.048, Test Accuracy: 98.660%
Epoch [6/10], Training Loss: 0.061, Training Accuracy: 98.217%, Test Loss: 0.058, Test Accuracy: 98.340%
Epoch [7/10], Training Loss: 0.066, Training Accuracy: 98.165%, Test Loss: 0.061, Test Accuracy: 98.400%
Epoch [8/10], Training Loss: 0.079, Training Accuracy: 97.960%, Test Loss: 0.074, Test Accuracy: 98.410%
Epoch [9/10], Training Loss: 0.079, Training Accuracy: 98.075%, Test Loss: 0.098, Test Accuracy: 97.890%
Epoch [10/10], Training Loss: 0.098, Training Accuracy:

In [ ]:
f = plt.figure(figsize = (10,10))
plt.plot(train_loss,label= "train_loss")
plt.plot(test_loss, label= "test_loss")
plt.legend()
plt.show()

In [ ]:
f = plt.figure(figsize = (10,10))
plt.plot(train_accuracy,label= "train_accuracy")
plt.plot(test_accuracy, label= "test_accuracy")
plt.legend()
plt.show()